In [1]:
# Update sklearn to prevent version mismatches
!conda install scikit-learn
!conda update scikit-learn
!conda install joblib 
!conda update joblib 

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1       False_Positive
2       False_Positive
3            CONFIRMED
4            CONFIRMED
             ...      
6986    False_Positive
6987    False_Positive
6988         CANDIDATE
6989    False_Positive
6990    False_Positive
Name: koi_disposition, Length: 6991, dtype: object

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [6]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4002,0,0,1,0,99.673478,3.463000e-04,-3.463000e-04,219.334830,0.002300,-0.002300,...,-148,4.777,0.040,-0.027,0.492,0.026,-0.027,293.05801,45.248821,15.801
4246,0,1,0,0,0.592244,9.000000e-08,-9.000000e-08,131.654831,0.000124,-0.000124,...,-146,4.664,0.056,-0.032,0.591,0.045,-0.045,290.28094,45.464260,15.653
548,0,1,1,0,9.991625,5.360000e-06,-5.360000e-06,137.447816,0.000445,-0.000445,...,-176,4.338,0.153,-0.187,1.096,0.309,-0.206,301.04239,45.022888,14.039
3953,0,1,0,0,178.412990,3.100000e-05,-3.100000e-05,218.225235,0.000127,-0.000127,...,-134,4.346,0.084,-0.126,1.148,0.202,-0.124,288.32785,38.627621,13.944
2362,0,0,0,0,45.294223,5.600000e-05,-5.600000e-05,138.678725,0.000987,-0.000987,...,-68,4.347,0.030,-0.030,1.044,0.057,-0.042,285.67938,50.241299,10.961


# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [7]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [10]:
y_train_categorical.shape

(5243, 3)

# Train the Model

In [11]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [14]:
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 0s - loss: 0.5401 - accuracy: 0.7328
Epoch 2/100
164/164 - 0s - loss: 0.3719 - accuracy: 0.7999
Epoch 3/100
164/164 - 0s - loss: 0.3515 - accuracy: 0.8240
Epoch 4/100
164/164 - 0s - loss: 0.3424 - accuracy: 0.8295
Epoch 5/100
164/164 - 0s - loss: 0.3369 - accuracy: 0.8316
Epoch 6/100
164/164 - 0s - loss: 0.3313 - accuracy: 0.8388
Epoch 7/100
164/164 - 0s - loss: 0.3254 - accuracy: 0.8478
Epoch 8/100
164/164 - 0s - loss: 0.3190 - accuracy: 0.8503
Epoch 9/100
164/164 - 0s - loss: 0.3151 - accuracy: 0.8495
Epoch 10/100
164/164 - 0s - loss: 0.3128 - accuracy: 0.8550
Epoch 11/100
164/164 - 0s - loss: 0.3057 - accuracy: 0.8552
Epoch 12/100
164/164 - 0s - loss: 0.3022 - accuracy: 0.8621
Epoch 13/100
164/164 - 0s - loss: 0.3076 - accuracy: 0.8556
Epoch 14/100
164/164 - 0s - loss: 0.2998 - accuracy: 0.8629
Epoch 15/100
164/164 - 0s - loss: 0.2901 - accuracy: 0.8707
Epoch 16/100
164/164 - 0s - loss: 0.2934 - accuracy: 0.8667
Epoch 17/100
164/164 - 0s - loss: 0.2897 - accura

164/164 - 0s - loss: 0.2483 - accuracy: 0.8894
Epoch 45/100
164/164 - 0s - loss: 0.2483 - accuracy: 0.8917
Epoch 46/100
164/164 - 0s - loss: 0.2458 - accuracy: 0.8926
Epoch 47/100
164/164 - 0s - loss: 0.2483 - accuracy: 0.8917
Epoch 48/100
164/164 - 0s - loss: 0.2527 - accuracy: 0.8899
Epoch 49/100
164/164 - 0s - loss: 0.2477 - accuracy: 0.8913
Epoch 50/100
164/164 - 0s - loss: 0.2438 - accuracy: 0.8930
Epoch 51/100
164/164 - 0s - loss: 0.2442 - accuracy: 0.8945
Epoch 52/100
164/164 - 0s - loss: 0.2438 - accuracy: 0.8961
Epoch 53/100
164/164 - 0s - loss: 0.2469 - accuracy: 0.8917
Epoch 54/100
164/164 - 0s - loss: 0.2449 - accuracy: 0.8957
Epoch 55/100
164/164 - 0s - loss: 0.2425 - accuracy: 0.8940
Epoch 56/100
164/164 - 0s - loss: 0.2404 - accuracy: 0.8961
Epoch 57/100
164/164 - 0s - loss: 0.2474 - accuracy: 0.8890
Epoch 58/100
164/164 - 0s - loss: 0.2494 - accuracy: 0.8915
Epoch 59/100
164/164 - 0s - loss: 0.2453 - accuracy: 0.8940
Epoch 60/100
164/164 - 0s - loss: 0.2411 - accuracy: 

164/164 - 0s - loss: 0.2234 - accuracy: 0.9031
Epoch 89/100
164/164 - 0s - loss: 0.2238 - accuracy: 0.9039
Epoch 90/100
164/164 - 0s - loss: 0.2303 - accuracy: 0.9016
Epoch 91/100
164/164 - 0s - loss: 0.2227 - accuracy: 0.9020
Epoch 92/100
164/164 - 0s - loss: 0.2239 - accuracy: 0.9014
Epoch 93/100
164/164 - 1s - loss: 0.2204 - accuracy: 0.9050
Epoch 94/100
164/164 - 0s - loss: 0.2237 - accuracy: 0.9010
Epoch 95/100
164/164 - 0s - loss: 0.2208 - accuracy: 0.9088
Epoch 96/100
164/164 - 0s - loss: 0.2206 - accuracy: 0.9064
Epoch 97/100
164/164 - 0s - loss: 0.2227 - accuracy: 0.9008
Epoch 98/100
164/164 - 0s - loss: 0.2173 - accuracy: 0.9050
Epoch 99/100
164/164 - 0s - loss: 0.2164 - accuracy: 0.9073
Epoch 100/100
164/164 - 0s - loss: 0.2169 - accuracy: 0.9086


In [15]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2918 - accuracy: 0.8850
Normal Neural Network - Loss: 0.291769415140152, Accuracy: 0.8850114345550537


In [16]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [17]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CANDIDATE' 'False_Positive' 'False_Positive' 'CANDIDATE'
 'False_Positive']
Actual Labels: ['CANDIDATE', 'False_Positive', 'False_Positive', 'CANDIDATE', 'False_Positive']


# Save the Model

In [18]:
# save fitted model to file
import joblib
filename = 'deep_learning.sav'
joblib.dump(model, filename)

TypeError: can't pickle _thread.RLock objects